## PARIS SUBSIDIES: DATA CLEANING (RECORDS)

### LIBRAIRIES IMPORT

In [1]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

### DATA IMPORT

In [3]:
subsidies = pd.read_csv('../00_DataFiles/01_Collected/ParisSubsidies_Records.csv')
subsidies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79973 entries, 0 to 79972
Data columns (total 10 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   Numéro de dossier                                79973 non-null  object
 1   Année budgétaire                                 79973 non-null  int64 
 2   Collectivité                                     79973 non-null  object
 3   Nom Bénéficiaire                                 79973 non-null  object
 4   Numéro Siret                                     79973 non-null  int64 
 5   Objet du dossier                                 79973 non-null  object
 6   Montant voté                                     79973 non-null  int64 
 7   Direction                                        79973 non-null  object
 8   Nature de la subvention                          79973 non-null  object
 9   Secteurs d'activités définies par l'ass

#### Profile report before cleaning

In [4]:
profile = ProfileReport(subsidies.reset_index(drop=True), title="Paris Subsidies Profiling Report - Before Cleaning")
profile.to_file("ProfileReports/ParisSubsidies_Records_bf.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 897.75it/s]


### FIELD BY FIELD APPROACH

### Missing values

In [5]:
subsidies.isna().sum()

Numéro de dossier                                   0
Année budgétaire                                    0
Collectivité                                        0
Nom Bénéficiaire                                    0
Numéro Siret                                        0
Objet du dossier                                    0
Montant voté                                        0
Direction                                           0
Nature de la subvention                             0
Secteurs d'activités définies par l'association    25
dtype: int64

#### Numéro de dossier
Despite high cardinality, we will keep the field so we can offer the possibility to retrieve a Paris City Council's deliberation from http://a06.apps.paris.fr/a06/jsp/site/Portal.jsp?page=search-solr

However, it should be unique for each record but some are present in two records, for instance '2019_07725'.

In [6]:
subsidies.loc[subsidies['Numéro de dossier'] == '2019_07725']

,Numéro de dossier,Année budgétaire,Collectivité,Nom Bénéficiaire,Numéro Siret,Objet du dossier,Montant voté,Direction,Nature de la subvention,Secteurs d'activités définies par l'association
40773,2019_07725,2019,Ville de Paris,FOODLAB,81850945700011,Un Foodlab rue de Mouzaia : Maison de l'alimen...,0,DASES,Non précisée,Education & formation;Précarité & Exclusion;Vi...
59337,2019_07725,2020,Ville de Paris,FOODLAB,81850945700011,Un Foodlab rue de Mouzaia : Maison de l'alimen...,100000,DASES,Investissement,Education & formation;Précarité & Exclusion;Vi...


Let's list all the seemingly duplicates

In [7]:
mask_duplicates = subsidies.duplicated(subset = 'Numéro de dossier', keep=False)
subsidies.loc[mask_duplicates].sort_values(by=['Numéro de dossier', 'Montant voté'])

,Numéro de dossier,Année budgétaire,Collectivité,Nom Bénéficiaire,Numéro Siret,Objet du dossier,Montant voté,Direction,Nature de la subvention,Secteurs d'activités définies par l'association
49587,2017_04250,2017,Ville de Paris,RED'S TEAM,50752562400015,Convention d'Objectifs,0,DJS,Non précisée,Education & formation;Social;Sport
60269,2017_04250,2020,Ville de Paris,RED S TEAM,50752562400015,EDUCATION PAR LE SPORT,16000,DJS,Fonctionnement,Education & formation;Social;Sport
31022,2018_00414,2018,Ville de Paris,MELTIN'CLUB PARIS,79109293500018,Fonctionnement 2017/2018,0,DJS,Fonctionnement,Humanitaire;Sport;Vie et animation locale
66573,2018_00414,2020,Ville de Paris,MELTIN'CLUB PARIS (M.C.P.),79109293500018,Fonctionnement 2020,7000,DJS,Projet,Humanitaire;Sport;Vie et animation locale
15095,2018_04293,2018,Ville de Paris,AGENCE PARISIENNE DU CLIMAT,52800732100011,Subvention DEVE-DLH-DU 2018,0,DU,Non précisée,Architecture & urbanisme;Déplacements et trans...
61,2018_04293,2019,Ville de Paris,AGENCE PARISIENNE DU CLIMAT (APC),52800732100011,solde 2019,81250,DEVE,Projet,Architecture & urbanisme;Déplacements et trans...
78652,2018_06960,2019,Ville de Paris,CERCLE DE TAEKWONDO DES ARCHERS,79805014200024,"Projets locaux de réussite éducative, appel à ...",0,DJS,Non précisée,Loisirs
58552,2018_06960,2020,Ville de Paris,CERCLE DE TAEKWONDO DES ARCHERS,79805014200024,"Projets locaux de réussite éducative, appel à ...",4000,SG,Projet,Loisirs
40360,2019_01028,2019,Ville de Paris,ATOUTPRISES,44806320600024,Renouvellement de voies des murs d'escalade du...,0,DJS,Non précisée,Sport;Vie et animation locale
62132,2019_01028,2020,Ville de Paris,ATOUTPRISES,44806320600024,Renouvellement de voies des murs d'escalade du...,2000,DJS,Fonctionnement,Sport;Vie et animation locale


In [8]:
# Save indices into list
mask_duplicates_index = subsidies.loc[mask_duplicates].index

# DataFrame to store dropped duplicates
df_duplicates = pd.DataFrame()

# Drop all lines with 'Montant voté' = 0 if they are a duplicate of a line with 'Montant voté' > 0
lst_duplicates_0 = list()


for i in mask_duplicates_index:
    j = subsidies.loc[subsidies['Numéro de dossier'] == subsidies.loc[i, 'Numéro de dossier']].index.to_list()
    j.remove(i)
    j = j[0]
    if subsidies.loc[i, 'Montant voté'] == 0 and subsidies.loc[j, 'Montant voté'] > 0:
        lst_duplicates_0.append(i)


df_duplicates = pd.concat([df_duplicates,subsidies.loc[lst_duplicates_0]], ignore_index=True)
subsidies.drop(lst_duplicates_0, axis = 0, inplace = True)

In [9]:
# Remaining duplicates
mask_duplicates = subsidies.duplicated(subset = 'Numéro de dossier', keep=False)
subsidies.loc[mask_duplicates].sort_values(by=['Numéro de dossier', 'Montant voté'])

,Numéro de dossier,Année budgétaire,Collectivité,Nom Bénéficiaire,Numéro Siret,Objet du dossier,Montant voté,Direction,Nature de la subvention,Secteurs d'activités définies par l'association
42834,2019_04634,2019,Ville de Paris,PLUS LOIN,39344280100048,Convention Espace de proximité DASES,41300,DJS,Non précisée,Loisirs;Social;Vie et animation locale
62493,2019_04634,2020,Ville de Paris,PLUS LOIN,39344280100048,subvention covid-19,41300,DJS,Projet,Loisirs;Social;Vie et animation locale
41462,2019_09377,2019,Ville de Paris,EMMAUS SOLIDARITE,31723624800017,Espace Femmes - ESI Agora,0,DASES,Non précisée,Humanitaire;Précarité & Exclusion;Social
71893,2019_09377,2021,Ville de Paris,EMMAUS SOLIDARITE,31723624800017,Espace Femmes - ESI Agora BUDGET PARTICIPATIF,0,DASES,Non précisée,Humanitaire;Précarité & Exclusion;Social
44919,2020_00258,2019,Ville de Paris,EIFFEL BASKET CLUB,44808025900014,Fonctionnement de l'association pour la saison...,2000,DJS,Fonctionnement,Sport
57826,2020_00258,2020,Ville de Paris,EIFFEL BASKET CLUB,44808025900022,Fonctionnement de l'association pour la saison...,2000,DJS,Fonctionnement,Sport
64611,2020_08294,2020,Ville de Paris,MAESTRA PRODUCTION,88148534600012,Création pièce de théâtre,0,DAC,Non précisée,Culture & Arts
73348,2020_08294,2021,Ville de Paris,MANIFEST CREATION (MC),88148534600020,Création pièce de théâtre,0,DAC,Non précisée,Culture & Arts
68655,2021_00774,2021,Ville de Paris,ASSOCIATION DE PREVENTION DU SITE DE LA VILLET...,34063517600019,Education à l'image à la Villette. Stage en mi...,28570,DPMP,Projet,Education & formation;Emploi;Précarité & Exclu...
60484,2021_00774,2020,Ville de Paris,ASSOCIATION DE PREVENTION DU SITE DE LA VILLET...,34063517600019,Education à l'image à la Villette. Stage en mi...,28591,DPSP,Projet,Education & formation;Emploi;Précarité & Exclu...


In [10]:
# Manual drop
lst_duplicates_1 = [62493, 41462, 44919, 73348, 60484]
df_duplicates = pd.concat([df_duplicates,subsidies.loc[lst_duplicates_1]], ignore_index=True)
subsidies.drop(lst_duplicates_1, axis = 0, inplace = True)
df_duplicates.to_csv('../00_DataFiles/99_Dropped/ParisSubsidies_Duplicates.csv')

### Année budgétaire
We keep the field

### Collectivité
We do not need the field: it used to make sense only before 2016 when Paris City Council was split between 'Mairie de Paris' and 'Département de Paris'

In [11]:
subsidies.drop('Collectivité', axis = 1, inplace=True)

### Nom Bénéficiaire
We will not use this field but instead take the name from the SIRENE database for consistency (prevents typos)

In [12]:
subsidies.drop('Nom Bénéficiaire', axis = 1, inplace=True)

### Numéro Siret
Essential field as uniquely identifies the beneficiairies and will be used as the key to join the SIRENE data

### Objet du dossier
To keep and to be treated with NLP tools

### Montant voté
Correctly identified as of integer type

### Direction
To keep for categorical analysis (not ordinal)

### Nature de la subvention
4 categories: Projet, Fonctionnement, Investissement, and Non Précisée

### Secteurs d'activités définies par l'association
String type but should be list

In [13]:
# First we need to fill NaNs
subsidies.loc[subsidies['Secteurs d\'activités définies par l\'association'].isna(), 'Numéro Siret'].unique()

array([45047976100030, 75170501300025, 79755853300015, 48469647100024,
       48209742500037, 85211596300013, 48467532700023, 78430883500015])

In [14]:
# Manual input
subsidies.loc[subsidies['Numéro Siret'] == 45047976100030, 'Secteurs d\'activités définies par l\'association'] = 'Culture & Arts'
subsidies.loc[subsidies['Numéro Siret'] == 75170501300025, 'Secteurs d\'activités définies par l\'association'] = 'Culture & Arts'
subsidies.loc[subsidies['Numéro Siret'] == 79755853300015, 'Secteurs d\'activités définies par l\'association'] = 'Social'
subsidies.loc[subsidies['Numéro Siret'] == 48469647100024, 'Secteurs d\'activités définies par l\'association'] = 'Culture & Arts'
subsidies.loc[subsidies['Numéro Siret'] == 48209742500037, 'Secteurs d\'activités définies par l\'association'] = 'Défense des droits et des intérêts'
subsidies.loc[subsidies['Numéro Siret'] == 85211596300013, 'Secteurs d\'activités définies par l\'association'] = 'Relations internationales'
subsidies.loc[subsidies['Numéro Siret'] == 48467532700023, 'Secteurs d\'activités définies par l\'association'] = 'Vie et animation locale'
subsidies.loc[subsidies['Numéro Siret'] == 78430883500015, 'Secteurs d\'activités définies par l\'association'] = 'Sport'


In [15]:
subsidies['Secteurs d\'activités définies par l\'association'] = subsidies['Secteurs d\'activités définies par l\'association'].str.split(';')

In [16]:
subsidies.head()

,Numéro de dossier,Année budgétaire,Numéro Siret,Objet du dossier,Montant voté,Direction,Nature de la subvention,Secteurs d'activités définies par l'association
0,2020_07586,2020,48905531900029,Co-financement DDCT d'un poste adultes-relais ...,4700,DDCT,Fonctionnement,"[Loisirs, Social, Vie et animation locale]"
1,2020_08053,2020,48905531900029,CS 2020/DASES SEPLEX,209692,DASES,Fonctionnement,"[Loisirs, Social, Vie et animation locale]"
2,2020_04885,2020,81147565600013,Ressourcerie Ephémère,0,DAE,Non précisée,"[Environnement & écologie, Sport, Vie et anima..."
3,2020_05032,2020,48909505900015,De Mômes et d'opérette : Des grottes de Lascau...,0,DASCO,Non précisée,"[Culture & Arts, Social, Sport]"
4,2020_02992,2020,34482479200032,Fonctionnement 2020 cité des ours 14 cité fala...,217134,DFPE,Fonctionnement,"[Education & formation, Santé, Social]"


## SAVE

### Profile report after cleaning

In [19]:
profile = ProfileReport(subsidies.reset_index(drop=True), title="Paris Subsidies Profiling Report - After Cleaning")
profile.to_file("ProfileReports/ParisSubsidies_Records_af.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 600.99it/s]


### CSV

In [18]:
subsidies.to_csv('../00_DataFiles/02_Cleaned/ParisSubsidies_Records.csv', index = False)